<a href="https://colab.research.google.com/github/serciex/lane-change/blob/main/replication_of_rl_lateral_controller_for_autonomous_vehicles_cartpole.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing Prerequisites

In [ ]:
!pip install pygame
!pip install highway-env

Testing

In [ ]:
#Import Libraries
from os import truncate
import math
import gymnasium
import highway_env
from matplotlib import pyplot as plt
import pygame
import numpy as np


from IPython.display import Image, display
from collections import deque
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
%matplotlib inline

Environment Definition

In [ ]:
# Configure Environment Conditions
config = {
    "lanes_count": 3,
    "lane_width": 3.75,
    "observation": {
        "type": "Kinematics",
        "features": ["x", "y", "vx", "vy", "heading", "lat_off"]
    },
    "action": {"type": "ContinuousAction"},"ego_spawn_random": True,
    "policy_frequency": 10,
}
env = gymnasium.make('highway-v0', render_mode='rgb_array', config=config)
frames = []

# Action Setup
highway_env.envs.common.action.ContinuousAction(env, lateral=True,
                                                longitudinal=True)

In [ ]:
# Environment Manager
class ENV(env.__class__):
  """
  s = (v,a,x,y,thetha,id,w,c) ∈ S

  Lateral Agent State:
  Obs Data:
  x = vehicle x position (x)
  y = vehicle y position (y)
  v = vehicle speed (vx)
  thetha = yaw angle (heading)

  Input:
  a = longitudinal acceleration (longitudinal_control)
  id = target lane id
  w = lane width
  c = road curvature

  Extra Data:
  vy = lateral rate (vy)
  delta_lat_deviation = change in lateral deviation (lat_off)

  """
  def __init__(self, obs,a):
    self.obs = obs
    self.a = a

  def ego_state_idm(self):
      ax, ay, avx, avy, athetha, _ = self.obs[0]
      self._ego_state_idm = {"x": ax, "y": ay, "vx": avx,"thetha": athetha,
                         "longitudinal_acceleration":self.a}

      return self._ego_state_idm

  def ego_state_agent(self,target_id):
      ax, ay, avx, avy, athetha, _ = self.obs[0]
      vehicle = env.unwrapped.vehicle

      # Control Parameters
      self.id = target_id

      # Environment Parameters
      self.s,_ = vehicle.lane.local_coordinates(vehicle.position)
      self.w = vehicle.lane.width
      self.c = vehicle.lane.heading_at(np.clip(
          vehicle.lane.local_coordinates(vehicle.position)[0],
          0, vehicle.lane.length))
      self.v = math.sqrt(avx**2+avy**2)

      self._ego_state_agent = {"x": ax, "y": ay, "vx": self.v,"thetha": athetha,
                         "lane_width":self.w,"lane_id":self.id,
                         "self_curvature":self.c,
                         "longitudinal_acceleration":self.a}

      return self._ego_state_agent

  def longitudinal_lead_state(self):
    # Lead State Parameters on the same Lane (5 columns: presence, x, y, vx, vy)
    ego_vehicle = env.unwrapped.vehicle
    lead_vehicle = env.unwrapped.road.neighbour_vehicles(ego_vehicle, lane_index=ego_vehicle.lane_index)[0]
    if lead_vehicle:
      gap = lead_vehicle.position[0] - ego_vehicle.position[0]
      delta_velocity = ego_vehicle.velocity[0] - lead_vehicle.velocity[0]
      self.longitudinal_lead_state = {"x": gap, "vx": delta_velocity}

    else:
      self.longitudinal_lead_state = {"x": 10, "vx": 0}

    return self.longitudinal_lead_state

  #Reward Function
  def reward_function(self, obs_old, obs_new, target_id, w1=1, w2=1, w3=0.05):
    """
    Reward Function:

    Acceleration Reward: r_acce = w1*f_acce(a_yaw)
    a_yaw = lateral acceleration (self.action)

    Rate Reward: r_rate = w2*f_rate(w_yaw)
    w_yaw = lateral rate (vy)

    Time Reward: r_time = w3*f_time (delta_lat_deviation)
    delta_lat_deviation = change in lateral deviation (self.lat_off)

    Reward = Cummulative Sum of r_acce + Cummulative Sum of r_rate + Cummulative Sum of r_time

    """

    self.target_id = ("0","1",target_id)
    target_lane_object = env.unwrapped.road.network.get_lane(self.target_id)
    vehicle_s, _ = env.unwrapped.vehicle.lane.local_coordinates(env.unwrapped.vehicle.position)
    _ , self.delta_lat_deviaton = target_lane_object.local_coordinates(env.unwrapped.vehicle.position)

    obs = obs_new[0]
    obs_old = obs_old[0]

    w_yaw = (obs[0] * obs[3] - obs[1] * obs[2]) / (obs[0]**2 + obs[1]**2+1e-8)
    w_yaw_old = (obs_old[0] * obs_old[3] - obs_old[1] * obs_old[2]) / (obs_old[0]**2 + obs_old[1])

    self.w_acce = (w_yaw-w_yaw_old)*env.unwrapped.config['policy_frequency']

    # Acceleration Reward
    acce_reward = -1*abs(self.w_acce)

    # Rate Reward
    rate_reward = -1*abs(w_yaw)

    # Time Reward
    time_reward = -0.05 * abs(self.delta_lat_deviaton)

    # Overall Reward
    self.reward = w1*acce_reward + w2*rate_reward + w3*time_reward

    return [self.reward, acce_reward, rate_reward, time_reward]

  #Acceleration to Steering angle
  def steering_angle(self, agent_action,L=1):
    """
    Steering Angle: theta = atan(a_yaw/v^2)
    a_yaw = lateral acceleration (agent_action)
    v = vehicle speed (vx)
    """
    self.angle = math.atan(L*agent_action/self.ego_state_idm()['vx']**2)

    return self.angle

Agent Defintion

In [ ]:
class Agent(nn.Module):
  """
  s = (x, y, vx, vy, thetha, lane_width, lane_id, self_curvature, longitudinal_acceleration) ∈ S

  Lateral Agent State:
  x = vehicle x position
  y = vehicle y position
  vx = vehicle speed (longitudinal)
  thetha = yaw angle (heading)
  lane_width = width of the lane
  lane_id = target lane id
  self_curvature = road curvature at the vehicle's position
  longitudinal_acceleration = vehicle longitudinal acceleration
  """
  def __init__(self, state_dim):
    super(Agent, self).__init__()
    self.state_dim = state_dim
    # Define Network A & C
    self.networkA = nn.Sequential(
        nn.Linear(self.state_dim, 100),
        nn.Linear(100, 1),
        nn.Softplus()
    )
    self.networkC = nn.Sequential(
        nn.Linear(self.state_dim + 1, 100),
        nn.Linear(100, 1),
    )

    # Define Network B
    self.networkB1 = nn.Sequential(
        nn.Linear(self.state_dim, 150),
        nn.Linear(150, 1),
    )
    self.networkB2 = nn.Sequential(
        nn.Linear(self.state_dim, 150),
        nn.Linear(150, 1),
    )
    self.networkB3 = nn.Sequential(
        nn.Linear(self.state_dim, 150),
        nn.Linear(150, 1),
    )

  def forward(self, state, a, terminal):
    """
    Q(s,a) = A(s)*(B(s)-a)^2 + C(s)
    """
    if isinstance(state, dict):
      state_tensor = torch.tensor(
          [state[key] for key in ['vx', 'longitudinal_acceleration', 'x', 'y', 'thetha', 'lane_id', 'lane_width', 'self_curvature']],
          dtype=torch.float32
      ).unsqueeze(0)
    else:
        self.state = state
    # Use a directly if it's already a tensor
    self.a = a if isinstance(a, torch.Tensor) else torch.tensor([[a]], dtype=torch.float32)
    self.terminal_condition = terminal if isinstance(terminal, torch.Tensor) else torch.tensor([[terminal]], dtype=torch.float32)

    # Output of the Networks
    self.A = self.networkA(self.state)
    self.C = self.networkC(torch.concat((self.state, self.terminal_condition), dim=1))
    self.B = torch.max(self.networkB1(self.state) * self.networkB2(self.state),
                         self.networkB3(self.state))

    # Q-function Approximation
    q_value =self.A * ((self.B - self.a) ** 2) + self.C

    return q_value

  def action(self, state):
      # Convert it to a tensor
      if isinstance(state, dict):
          state_tensor = torch.tensor(
              [state[key] for key in ['vx', 'longitudinal_acceleration', 'x', 'y', 'thetha', 'lane_id', 'lane_width', 'self_curvature']],
              dtype=torch.float32
          ).unsqueeze(0)
      else:
          state_tensor = state
      self.state = state_tensor
      self.B = torch.max(self.networkB1(self.state) * self.networkB2(self.state),
                         self.networkB3(self.state))
      return self.B

Experience Buffer

In [ ]:
# Buffer Class
class Experience_Buffer():
  """
  Define Experience Buffer
  """
  def __init__(self, buffer_size):
    self.buffer_size = buffer_size
    # Define Buffer as Deque with Limit of Default Size buffer_size elements
    self.buffer = deque(maxlen=self.buffer_size)

  def add(self, state, action, reward, next_state, terminal_condition):
    # Experience is the state transition tensor/vector
    state_tensor = torch.tensor(
        [state[key] for key in ['vx', 'longitudinal_acceleration', 'x', 'y', 'thetha', 'lane_id', 'lane_width', 'self_curvature']],
        dtype=torch.float32
    ).unsqueeze(0)
    next_state_tensor = torch.tensor(
        [next_state[key] for key in ['vx', 'longitudinal_acceleration', 'x', 'y', 'thetha', 'lane_id', 'lane_width', 'self_curvature']],
        dtype=torch.float32
    ).unsqueeze(0)
    action_tensor = torch.tensor([[action]], dtype=torch.float32)
    reward_tensor = torch.tensor([[reward]], dtype=torch.float32)
    terminal_tensor = torch.tensor([[terminal_condition]], dtype=torch.float32)

    self.transition = torch.concat((state_tensor, action_tensor, reward_tensor,
                                    next_state_tensor, terminal_tensor), dim=1)
    self.buffer.append(self.transition)

  def sample_experience(self, batch_size):
    # Randomly sample Experience from buffer
    batch = random.sample(self.buffer, batch_size)
    return batch

  def size(self):
    # Return the current number of elements in the buffer.
    return len(self.buffer)


Lateral Controller (Gap Checker)

In [ ]:
import numpy as np

class Gap_Controller(env.__class__):
    def __init__(self, obs=None, following_gap_threshold=10.0):
        # Optionally store an initial observation if provided.
        if obs is not None:
            self.obs = obs
        # Threshold for safety gap with the following vehicle
        self.following_gap_threshold = following_gap_threshold

    def lane_checker(self):
        """
        Determines which adjacent lane offers the safest gap for a lane change.
        It examines the current lane, the lane to the left, and the lane to the right,
        returning the lane index that has the largest safe front gap available.
        In addition, if the gap with the following vehicle in the target lane is below the safety
        threshold, the lane change is aborted and the current lane is chosen.

        Returns:
            int: The target lane index (the lane with the largest safe gap, or the current lane)
                 if the candidate lane's following vehicle gap is unsafe.
        """
        lane_number = len(env.unwrapped.road.network.lanes_list())
        ego = env.unwrapped.vehicle

        # Initialize front gap values with a default that indicates an unsafe or non-existent gap.
        gap_current_front = -float('inf')
        gap_right_front = -float('inf')
        gap_left_front = -float('inf')

        # Initialize following gap values (gap between ego and the vehicle behind)
        gap_current_follower = float('inf')
        gap_right_follower = float('inf')
        gap_left_follower = float('inf')

        # Get vehicle neighbours in current lane:
        neighbours_current = env.unwrapped.road.neighbour_vehicles(ego, ego.lane_index)
        # neighbours_current[0] is the vehicle ahead and [1] is the following vehicle.
        if neighbours_current:
            if neighbours_current[0]:
                gap_current_front = neighbours_current[0].position[0] - ego.position[0]
            if len(neighbours_current) > 1 and neighbours_current[1]:
                gap_current_follower = ego.position[0] - neighbours_current[1].position[0]

        # Compute the left and right lane indices.
        current_lane = list(ego.lane_index)
        if current_lane[2] > 0:
            lane_left = (current_lane[0], current_lane[1], current_lane[2] - 1)
        else:
            lane_left = ego.lane_index

        if current_lane[2] < (lane_number - 1):
            lane_right = (current_lane[0], current_lane[1], current_lane[2] + 1)
        else:
            lane_right = ego.lane_index

        # Retrieve neighbour vehicles for the right lane.
        neighbours_right = env.unwrapped.road.neighbour_vehicles(ego, lane_index=lane_right)
        if neighbours_right:
            if neighbours_right[0]:
                gap_right_front = neighbours_right[0].position[0] - ego.position[0]
            if len(neighbours_right) > 1 and neighbours_right[1]:
                gap_right_follower = ego.position[0] - neighbours_right[1].position[0]

        # Retrieve neighbour vehicles for the left lane.
        neighbours_left = env.unwrapped.road.neighbour_vehicles(ego, lane_index=lane_left)
        if neighbours_left:
            if neighbours_left[0]:
                gap_left_front = neighbours_left[0].position[0] - ego.position[0]
            if len(neighbours_left) > 1 and neighbours_left[1]:
                gap_left_follower = ego.position[0] - neighbours_left[1].position[0]

        # Compare the front gaps: current, right, and left.
        # The candidate lane is chosen based on the largest front gap.
        front_gaps = [gap_current_front, gap_right_front, gap_left_front]
        best_index = np.argmax(front_gaps)

        # Determine the target lane index based on the best candidate.
        # best_index: 0 => current lane, 1 => right lane, 2 => left lane.
        if best_index == 1:
            candidate_lane = lane_right
            candidate_follower_gap = gap_right_follower
        elif best_index == 2:
            candidate_lane = lane_left
            candidate_follower_gap = gap_left_follower
        else:
            candidate_lane = ego.lane_index
            candidate_follower_gap = gap_current_follower  # in current lane, we don't enforce follower gap condition

        # Check if the candidate lane (if different from the current lane)
        # meets the follower gap condition.
        if candidate_lane != ego.lane_index:
            if candidate_follower_gap < self.following_gap_threshold:
                # The follower gap is too small; do not change lanes.
                target_lane_id = ego.lane_index[2]
            else:
                target_lane_id = candidate_lane[2]
        else:
            target_lane_id = ego.lane_index[2]

        return target_lane_id


Longitudinal Controller

In [ ]:
class IDM():
    ''' Intelligent Driving Model for Longitudinal Control

    Control parameters:
      a: maximum acceleration
      b: comfortable deceleration
      delta: acceleration exponent
      T: safe time headway

    Parameters:
      s0: minimum gap
      v0: desired speed

      a (Maximum Acceleration): How fast the vehicle can speed up (m/s²).
      b (Comfortable Deceleration): How smoothly the vehicle slows down (m/s²).
      δ (Acceleration Exponent): The non-linearity factor in acceleration.
      T (Safe Time Headway): The desired minimum following time gap (s).

    Input variables:
      s: current gap
      v: current vehicle speed
      delta_v: relative speed (difference between the vehicle's speed and the leading vehicle's speed)
    '''

    def __init__(self, desired_parameters, control_parameters):
        # Unpack initial parameters: [s0, v0]
        self.s0, self.v0 = desired_parameters
        # Unpack control parameters: [a, b, delta, T]
        self.a, self.b, self.delta, self.T = control_parameters

    def longitudinal_controller(self, input_variables):
        # Unpack input variables: [s, v, delta_v]
        s, v, delta_v = input_variables

        # Small epsilon to account for very small gaps and avoid division by zero
        epsilon = 1e-6

        # Desired gap: s* = s0 + v*T + (v * delta_v) / (2 * sqrt(a * b))
        desired_gap = self.s0 + max(0, v * self.T + ((v * delta_v) / (2 * math.sqrt(self.a * self.b))))

        # IDM acceleration: a_IDM = a * [ 1 - (v / v0)^delta - (s* / s)^2 ]
        acceleration = self.a * (1 - (v / self.v0)**self.delta - (desired_gap / (s + epsilon))**2)

        return acceleration

Initialize Environment

In [ ]:
# Initialize Environment
obs, _ = env.reset()

# Initialize Environment Manager and Reward
state_manager = ENV(obs,obs[0][2])

# State Manager for Ego and Lead State
ego_state_idm = state_manager.ego_state_idm()
lead_state = state_manager.longitudinal_lead_state()

# Initial Longitundinal Positions
ego_position_idm = ego_state_idm['x']
lead_position_idm = lead_state['x']

# Initial Velocities (using vx for longitudinal control)
ego_velocity_idm = ego_state_idm['vx']
lead_velocity_idm = lead_state['vx']

Training Loop

CartPole (Test)

In [ ]:
# ────────────────────────────────  ENV  ───────────────────────────────────────
import gymnasium
env_name          = "CartPole-v1"
env               = gymnasium.make(env_name, render_mode=None)
state_dim         = env.observation_space.shape[0]      # 2  (pos, vel)
act_low, act_high = 0, env.action_space.n - 1
device            = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ─────────────────────────────  HYPERPARAMETERS  ───────────────────────────────────────
gamma                 = 0.95
batch_size            = 10000
update_per_n_steps    = 10000
num_episodes          = 10000
max_steps_per_episode = 999
lr                    = 1e-2
replay_buffer_size    = batch_size*5
min_replay_size       = batch_size*2
plot_every            = 10

# --- Epsilon-Greedy Parameters ---
epsilon_start         = 0.99
epsilon_end           = 0.05
epsilon_decay_steps   = 7500
epsilon_decay_rate    = (epsilon_start - epsilon_end) / epsilon_decay_steps
epsilon               = epsilon_start

episode_returns = []
loss_history    = []
global_step     = 0

# ─────────────────────────────  NETWORKS  ─────────────────────────────────────
policy_net = Agent(state_dim).to(device)
target_net = Agent(state_dim).to(device)

target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer  = optim.Adam(policy_net.parameters(), lr=lr)
loss_fn    = nn.MSELoss()

# ───────────────────────────  REPLAY BUFFER  ─────────────────────────────────
class ReplayBuf:
    def __init__(self, capacity):
        self.buf = deque(maxlen=capacity)

    def add(self, s, a, r, s2, d):
        s_tensor = s.squeeze(0).cpu()
        s2_tensor = s2.squeeze(0).cpu()
        self.buf.append((s_tensor, a, r, s2_tensor, d))

    def sample(self, n):
        if len(self.buf) < n: return None
        batch = random.sample(self.buf, n)

        s, a, r, s2, d = zip(*batch)
        s_batch = torch.stack(s).to(device)
        a_batch = torch.tensor(a, dtype=torch.float32, device=device).unsqueeze(1)
        r_batch = torch.tensor(r, dtype=torch.float32, device=device).unsqueeze(1)
        s2_batch = torch.stack(s2).to(device)
        d_batch = torch.tensor(d, dtype=torch.float32, device=device).unsqueeze(1)
        return s_batch, a_batch, r_batch, s2_batch, d_batch

    def __len__(self):
        return len(self.buf)

replay = ReplayBuf(replay_buffer_size)

# ─────────────────────────── TRAINING LOOP ────────────────────────────────────
for ep in range(1, num_episodes + 1):
    obs, _ = env.reset()
    state  = torch.tensor(obs, dtype=torch.float32).unsqueeze(0).to(device)
    done   = False
    steps  = 0
    ep_ret = 0.0
    last_step_loss = None
    policy_net.train()

    while not done and steps < max_steps_per_episode:
        steps       += 1
        global_step += 1

        # Epsilon-greedy action selection
        if random.random() < epsilon:
            action_val = random.randint(0, 1)
        else:
            with torch.no_grad():
                terminal_tensor = torch.tensor([[0.0]], device=device)
                q_value_0 = policy_net(state, torch.tensor([[0.0]], device=device), terminal_tensor)
                q_value_1 = policy_net(state, torch.tensor([[1.0]], device=device), terminal_tensor)
                q_values = torch.cat((q_value_0, q_value_1), dim=1)
            action_val = torch.argmax(q_values).item()

        # Step environment
        obs2, reward, terminated, truncated, _ = env.step(action_val)
        done       = terminated or truncated
        ep_ret    += reward

        next_state = torch.tensor(obs2, dtype=torch.float32).unsqueeze(0).to(device)
        replay.add(state, float(action_val), float(reward), next_state, float(done))
        state = next_state

        # Learning step
        if len(replay) >= min_replay_size:
            sample = replay.sample(batch_size)
            if sample is not None:
                s_b, a_b, r_b, s2_b, d_b = sample
                q_curr_b = policy_net(s_b, a_b, d_b)

                with torch.no_grad():
                    q_next_val = target_net.networkC(torch.concat((s2_b, d_b), dim=1))
                    q_target_b = r_b + gamma * q_next_val * (1 - d_b)

                loss = loss_fn(q_curr_b, q_target_b)
                last_step_loss = loss.item()
                loss_history.append(loss.item())
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                # Decay epsilon after training update
                epsilon = max(epsilon_end, epsilon - epsilon_decay_rate)

        # Update target network
        if global_step % update_per_n_steps == 0:
            target_net.load_state_dict(policy_net.state_dict())

        if done:
            break

    # Track returns and print
    episode_returns.append(ep_ret)
    avg_return = np.mean(episode_returns[-100:])
    loss_str_repr = f"{last_step_loss:.4f}" if last_step_loss is not None else "N/A"

    print(f"Ep {ep:4}/{num_episodes} │ Steps:{steps:4} │ Ep Return:{ep_ret:8.2f} "
          f"│ Avg Return (100 ep):{avg_return:8.2f} │ Epsilon:{epsilon:.3f} "
          f"│ Buffer:{len(replay):6} │ Loss:{loss_str_repr}")

# Plot results
plt.plot(episode_returns)
plt.xlabel("Episode")
plt.ylabel("Return")
plt.title("Episode Returns over Time")
plt.show()

plt.plot(loss_history)
plt.xlabel("Step")
plt.ylabel("Loss")
plt.title("Training Loss over Time")
plt.show()

env.close()


Always Active (Option 1)

In [ ]:
################################################################################
"""Initialization"""
################################################################################
# IDM Initialization
control_parameters = [0.1, 5, 4, 4]  # a, b, δ, T
desired_parameters = [20, 10.0]       # s0, v0

# Set initial Input variables using the gap, current velocity, and relative lead velocity
input_variables = [lead_position_idm, ego_velocity_idm, lead_velocity_idm]

# Setup the IDM Model for Longitudinal control
ego_vehicle_idm = IDM(desired_parameters, control_parameters)

# Agent Initialization
total_steps_taken = 0
lr = 0.01
gamma = 0.9

buffer_size = int(1e+5)
num_episodes = int(2e+5)
batch_size = int(1e+4)
timesteps = int(1e+7)

update_target_frequency = int(1e+4)
threshold = 30
plot_frequency = 100

# Epsilon-Greedy Parameters
epsilon_start = 1.0
epsilon_end = 0.05
epsilon_decay_steps = int(1e+5)
epsilon = epsilon_start

# Tracking for performance
episode_returns = []
global_step = 0

target_id = env.unwrapped.vehicle.lane_index[-1]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Environment Information
L = env.unwrapped.vehicle.LENGTH
agent_state = state_manager.ego_state_agent(target_id=target_id)
state_dim = len(agent_state)

# Initialize Buffer
buffer = Experience_Buffer(buffer_size)

# Setup Lateral Control Agent
target_network = Agent(state_dim).to(device)
policy_network = Agent(state_dim).to(device)

# Initialize target network
target_network.load_state_dict(policy_network.state_dict())

# Loss and Criterion Initialization
policy_optimizer = torch.optim.Adam(policy_network.parameters(), lr=lr)
policy_loss_fn = nn.MSELoss()

# Tracking loss values and timestep rewards over training steps
loss_history = []
timestep_rewards = []

# Flag to indicate when training has started
training_started = False

# Lists for plotting data that will only be populated after training starts
plot_episode_actions = []
plot_timestep_rewards = []

################################################################################
"""Training & Testing Loop"""
################################################################################
ep = 0
while global_step < timesteps:
    # Reset environment and state manager
    obs, _ = env.reset()
    state_manager = ENV(obs, 0.0)

    # Initial state information
    ego_state_idm = state_manager.ego_state_idm()
    lead_state = state_manager.longitudinal_lead_state()
    agent_state = state_manager.ego_state_agent(target_id=target_id)

    # Ensure on_road_check is initialized correctly
    on_road_check = env.unwrapped.vehicle.on_road

    on_road_check = True
    collision_check = False

    # Agent activation flag and target lane storage
    activated_target_lane = None

    # Update IDM inputs based on initial state
    gap = lead_state['x']
    delta_velocity = lead_state['vx']
    input_variables = [gap, ego_state_idm['vx'], delta_velocity]

    steps = 0
    terminal = 0
    ep_ret = 0.0

    # For stepwise plotting, record agent actions per timestep
    episode_actions = []

    # Main loop for each episode
    while terminal == 0 and on_road_check:
        steps += 1
        global_step += 1  # This increments the global step counter for each environment step

        # Recompute gap
        gap = lead_state['x']

        ########################################################################
        """Action Preparation"""
        ########################################################################
        gap_control = Gap_Controller(obs, threshold)
        activated_target_lane = gap_control.lane_checker()

        # Determine target lane
        target_id = activated_target_lane
        agent_state = state_manager.ego_state_agent(target_id=target_id)

        with torch.no_grad():
            state_vector = torch.tensor(
                list(agent_state.values()),
                dtype=torch.float32,
                device=device
            ).unsqueeze(0)
            agent_action = policy_network.action(state_vector).item()

        # Store prior state for buffer
        old_state = agent_state
        obs_old = obs

        # IDM Longitudinal Control
        idm_acceleration = ego_vehicle_idm.longitudinal_controller(input_variables)
        longitudinal_control = idm_acceleration

        # Record the action for later stepwise plotting if training has started
        if training_started:
            episode_actions.append(agent_action)

        # Transform agent action to steering angle
        lateral_control = state_manager.steering_angle(agent_action, L)

        # Combine longitudinal and lateral actions
        action = [longitudinal_control, lateral_control]

        ########################################################################
        """Data Collection"""
        ########################################################################
        obs, reward, done, truncated, info = env.step(action)

        # Update state manager with new observation and applied longitudinal control
        state_manager = ENV(obs, longitudinal_control)

        # Compute reward based on the new state
        reward_per_episode = state_manager.reward_function(obs_old, obs, target_id)
        reward = reward_per_episode[0]
        ep_ret += reward

        # Append the reward from the current timestep only if training has started
        if training_started:
            plot_timestep_rewards.append(reward_per_episode)

        # For tracking overall performance, regardless of training
        timestep_rewards.append(reward_per_episode)

        # Terminal Condition Check
        if steps == timesteps:
            terminal = 1

        # Check if vehicle is off road
        on_road_check = env.unwrapped.vehicle.on_road

        ########################################################################
        """Update IDM Controller Inputs for next step"""
        ########################################################################
        ego_state_idm = state_manager.ego_state_idm()
        lead_state = state_manager.longitudinal_lead_state()
        agent_state = state_manager.ego_state_agent(target_id=target_id)

        gap = lead_state['x']
        delta_velocity = lead_state['vx']
        input_variables = [gap, ego_state_idm['vx'], delta_velocity]

        # Update experience buffer
        buffer.add(old_state, agent_action, reward, agent_state, terminal)

        ########################################################################
        """Q-Learning Update"""
        ########################################################################
        if buffer.size() >= batch_size:
            # Set training flag to true if it hasn't been set yet
            if not training_started:
                training_started = True
                # Reset plot data collection lists when training starts
                plot_episode_actions = []
                plot_timestep_rewards = []
                print("Training started - beginning to collect data for plotting")

            rand_experience = buffer.sample_experience(batch_size=batch_size)
            transitions = torch.cat(
                [t for t in rand_experience if isinstance(t, torch.Tensor)],
                dim=0
            ).to(device)

            states, actions_t, rewards_t, next_states, terminals_t = torch.split(
                transitions, [state_dim, 1, 1, state_dim, 1], dim=1
            )
            current_q_values = policy_network(states, actions_t, terminals_t)

            with torch.no_grad():
                target_network.eval()
                next_state = torch.cat((next_states, terminals_t), dim=1)
                max_next_q_values = target_network.networkC(next_state)

            target_q_values = rewards_t + gamma * max_next_q_values * (1.0 - terminals_t)
            loss = policy_loss_fn(current_q_values, target_q_values)
            loss_history.append(loss.item())

            policy_optimizer.zero_grad()
            loss.backward()
            policy_optimizer.step()

            total_steps_taken += 1

        # Update epsilon linearly over decay steps - only once training has started
        if buffer.size() >= batch_size:
            epsilon = max(
                epsilon_end,
                epsilon_start - (total_steps_taken * (epsilon_start - epsilon_end) / epsilon_decay_steps)
            )

    ############################################################################
    """Update Target Policy Network"""
    ############################################################################
    if ep % update_target_frequency == 0:
        target_network.load_state_dict(policy_network.state_dict())
        print(f"**** Target network updated at episode {ep} ****")

    # Print training progress
    episode_returns.append(ep_ret)
    avg_return = np.mean(episode_returns[-100:])
    last_step_loss = loss_history[-1] if loss_history else None
    loss_str_repr = f"{last_step_loss:.4f}" if last_step_loss is not None else "N/A"
    print(f"Ep {ep:4} │ Global Steps:{global_step:6}/{timesteps} │ Ep Return:{ep_ret:8.2f} │ Avg Return (100 ep):{avg_return:8.2f} │ Epsilon:{epsilon:.3f} │ Buffer:{buffer.size():6} │ Loss:{loss_str_repr}")

    # Increment episode counter
    ep += 1

    ############################################################################
    """Periodic Plotting of Reward and Loss after DQN Update (every x timesteps)"""
    ############################################################################
    if global_step % plot_frequency == 0 and global_step > 0 and training_started:
        if len(plot_episode_actions) > 0:
            plt.figure(figsize=(10, 6))
            plt.plot(range(len(plot_episode_actions)), plot_episode_actions, label="Steering Action (a_yaw)")
            plt.xlabel("Steps Since Training Started")
            plt.ylabel(f"Agent Action per Step (Timestep {global_step})")
            plt.legend()
            plt.title("Agent Action per Step")
            plt.tight_layout()
            plt.show()

        if len(plot_timestep_rewards) > 0:
            plt.figure(figsize=(12, 5))
            plt.plot(range(len(plot_timestep_rewards)), plot_timestep_rewards, label=['R_Total','R_acce','R_rate','R_time'])
            plt.xlabel("Steps Since Training Started")
            plt.ylabel("Reward")
            plt.title("Reward per Timestep")
            plt.legend()
            plt.tight_layout()
            plt.show()

        if len(loss_history) > 0:
            plt.figure()
            plt.plot(range(len(loss_history)), loss_history)
            plt.xlabel("Q-Learning Update")
            plt.ylabel("Loss")
            plt.title("Loss over Training Updates")
            plt.show()

**** Target network updated at episode 0 ****
Ep    0 │ Global Steps:     8/10000000 │ Ep Return:   -5.68 │ Avg Return (100 ep):   -5.68 │ Epsilon:1.000 │ Buffer:     8 │ Loss:N/A
Ep    1 │ Global Steps:    11/10000000 │ Ep Return:   -1.61 │ Avg Return (100 ep):   -3.65 │ Epsilon:1.000 │ Buffer:    11 │ Loss:N/A
Ep    2 │ Global Steps:    16/10000000 │ Ep Return:   -3.21 │ Avg Return (100 ep):   -3.50 │ Epsilon:1.000 │ Buffer:    16 │ Loss:N/A
Ep    3 │ Global Steps:    19/10000000 │ Ep Return:   -1.63 │ Avg Return (100 ep):   -3.03 │ Epsilon:1.000 │ Buffer:    19 │ Loss:N/A
Ep    4 │ Global Steps:    24/10000000 │ Ep Return:   -3.19 │ Avg Return (100 ep):   -3.07 │ Epsilon:1.000 │ Buffer:    24 │ Loss:N/A
Ep    5 │ Global Steps:    32/10000000 │ Ep Return:   -5.71 │ Avg Return (100 ep):   -3.51 │ Epsilon:1.000 │ Buffer:    32 │ Loss:N/A
Ep    6 │ Global Steps:    35/10000000 │ Ep Return:   -1.60 │ Avg Return (100 ep):   -3.23 │ Epsilon:1.000 │ Buffer:    35 │ Loss:N/A
Ep    7 │ Global

Inactive Agent Until Lane (Option 2)

In [ ]:
################################################################################
"""Initialization"""
################################################################################
# IDM Initialization
control_parameters = [0.1, 5, 4, 4]  # a, b, δ, T
desired_parameters = [40, 10.0]       # s0, v0

# Set initial Input variables using the gap, current velocity, and relative lead velocity
input_variables = [lead_position_idm, ego_velocity_idm, lead_velocity_idm]

# Setup the IDM Model for Longitudinal control
ego_vehicle_idm = IDM(desired_parameters, control_parameters)

# Agent Initialization
total_steps_taken = 0
lr = 0.01
gamma = 0.9
buffer_size = 1000
num_episodes = 10000
batch_size = 500
timesteps = 40000
update_target_frequency = 10
# epsilon_decay_rate = 10000
threshold = 30
plot_frequency = 10
threshold = 30
thetha_threshold = 0.1

target_id = env.unwrapped.vehicle.lane_index[-1]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Environment Information
L = env.unwrapped.vehicle.LENGTH
agent_state = state_manager.ego_state_agent(target_id=target_id)
state_dim = len(agent_state)

# Initialize Buffer
buffer = Experience_Buffer(buffer_size)

# Setup Lateral Control Agent
target_network = Agent(state_dim).to(device)
policy_network = Agent(state_dim).to(device)

# Initialize target network
target_network.load_state_dict(policy_network.state_dict())

# Loss and Criterion Initialization
policy_optimizer = torch.optim.Adam(policy_network.parameters(), lr=lr)
policy_loss_fn = nn.MSELoss()

# Tracking loss values and storing per-timestep rewards for plotting.
loss_history = []
# We remove the per-episode accumulation.
# episode_rewards = []
timestep_rewards = []  # New list: store each timestep's reward directly.

################################################################################
"""Training & Testing Loop"""
################################################################################
for episodes in range(num_episodes):
    # Reset environment and state manager
    obs, _ = env.reset()
    state_manager = ENV(obs, 0.0)

    # Initial state information
    ego_state_idm = state_manager.ego_state_idm()
    lead_state = state_manager.longitudinal_lead_state()
    agent_state = state_manager.ego_state_agent(target_id=target_id)

    on_road_check = True
    collision_check = False

    # Agent activation flag and target lane storage
    activated_target_lane = None

    # Update IDM inputs based on initial state
    gap = lead_state['x']
    delta_velocity = lead_state['vx']
    input_variables = [gap, ego_state_idm['vx'], delta_velocity]

    steps = 0
    terminal = 0
    # Removed cumulative episode reward accumulation.
    # current_episode_reward = 0
    # episode_acce_reward = 0
    # episode_rate_reward = 0
    # episode_time_reward = 0
    counter = 0
    # epsilon = 0.1

    # For stepwise plotting, record agent actions per timestep
    episode_actions = []

    # Main loop for each episode
    while steps < timesteps and counter < 1:
        steps += 1
        # Epsilon decay based on total_steps_taken
        # epsilon = max(0.05, 0.1 - (0.1 - 0.05) * (total_steps_taken / epsilon_decay_rate))

        # Recompute gap
        gap = lead_state['x']

        ########################################################################
        """Action Preparation"""
        ########################################################################
        gap_control = Gap_Controller(obs, threshold)
        activated_target_lane = gap_control.lane_checker()

        # Determine target lane
        target_id = activated_target_lane
        agent_state = state_manager.ego_state_agent(target_id=target_id)

        with torch.no_grad():
            # Agent Remains Inactive Unless Heading and Gap Meet a Requirement
            if (gap < threshold and target_id != env.unwrapped.vehicle.lane_index[2]) or abs(agent_state['thetha']) > thetha_threshold:
                agent_action = policy_network.action(agent_state).item()
            else:
                agent_action = 0

        # Store prior state for buffer
        old_state = agent_state
        obs_old = obs

        # IDM Longitudinal Control
        idm_acceleration = ego_vehicle_idm.longitudinal_controller(input_variables)
        longitudinal_control = idm_acceleration

        # Record the action for later stepwise plotting (e.g., steering angle per timestep)
        episode_actions.append(agent_action)

        # Transform agent action (acceleration prediction) to steering angle
        steering_angle = state_manager.steering_angle(agent_action, L)
        lateral_control = steering_angle

        # Combine longitudinal and lateral actions
        action = [longitudinal_control, lateral_control]

        ########################################################################
        """Data Collection"""
        ########################################################################
        obs, reward, done, truncated, info = env.step(action)

        # Check for off-road or collision events
        on_road_check = env.unwrapped.vehicle.on_road
        collision_check = info['crashed']

        # Update state manager with new observation and applied longitudinal control
        state_manager = ENV(obs, longitudinal_control)
        # Compute reward based on the new state
        reward_per_episode = state_manager.reward_function(obs_old, obs, target_id)
        # Grab the reward (the first element of the returned list)
        reward = reward_per_episode[0]

        # Instead of adding up rewards per episode, simply append the reward from this timestep.
        timestep_rewards.append(reward)

        # Removed cumulative sums:
        # current_episode_reward += reward
        # episode_acce_reward += reward_per_episode[1]
        # episode_rate_reward += reward_per_episode[2]
        # episode_time_reward += reward_per_episode[3]

        ########################################################################
        """Update IDM Controller Inputs for next step"""
        ########################################################################
        ego_state_idm = state_manager.ego_state_idm()
        lead_state = state_manager.longitudinal_lead_state()
        agent_state = state_manager.ego_state_agent(target_id=target_id)

        gap = lead_state['x']
        delta_velocity = lead_state['vx']
        input_variables = [gap, ego_state_idm['vx'], delta_velocity]

        ########################################################################
        """Terminal Condition Check"""
        ########################################################################
        # Update error counter
        if not on_road_check or collision_check:
            counter += 1
            terminal = 1

        # Terminal condition check: if steps reaches timesteps, mark terminal.
        if steps == timesteps:
            terminal = 1
        else:
            terminal = 0

        # Update experience buffer
        buffer.add
